# Document Similarity

# Rekomendasi Film dengan Kemiripan Dokumen

Sistem rekomendasi adalah salah satu aplikasi pembelajaran mesin yang populer dan paling banyak diadopsi. Biasanya digunakan untuk merekomendasikan entitas kepada pengguna dan entitas ini dapat berupa apa saja seperti produk, film, layanan, dan sebagainya.

Contoh rekomendasi yang populer meliputi,
- Amazon menyarankan produk di situs webnya
- Amazon Prime, Netflix, Hotstar merekomendasikan film\acara
- YouTube merekomendasikan video untuk ditonton

Biasanya sistem pemberi rekomendasi dapat diimplementasikan dalam tiga cara:

- Simple Rule-based Recommenders / Rekomendasi Berbasis Aturan Sederhana: Biasanya berdasarkan metrik dan ambang batas global tertentu seperti popularitas film, peringkat global, dll.
- Content-based Recommenders / Rekomendasi berbasis konten: Ini didasarkan pada penyediaan entitas serupa berdasarkan entitas minat tertentu. Metadata konten dapat digunakan di sini seperti deskripsi film, genre, pemeran, sutradara, dan sebagainya
- Collaborative filtering Recommenders / Pemfilteran kolaboratif Rekomendasi: Di ​​sini tidak diperlukan metadata, tetapi memprediksi rekomendasi dan peringkat berdasarkan peringkat sebelumnya dari berbagai pengguna dan item tertentu.

STKI / NLP paling relevan dengan Collaborative filtering Recommenders

## install library

In [1]:
# !pip install textsearch
# !pip install contractions
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')

In [2]:
import pathlib
pathlib.Path().resolve()

WindowsPath('C:/Users/ACER')

## load dan view data

In [3]:
import pandas as pd

df = pd.read_csv('/Users/masaboe/Documents/NGAJAR/2022/Ganjil 2/AMS-SI/Materi/Lab/docSimilarity/tmdb_5000_movies.csv.gz', compression='gzip')
df.info()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/masaboe/Documents/NGAJAR/2022/Ganjil 2/AMS-SI/Materi/Lab/docSimilarity/tmdb_5000_movies.csv.gz'

In [ ]:
df.head()

In [ ]:
df = df[['title', 'tagline', 'overview', 'popularity']]
df.tagline.fillna('', inplace=True)
df['description'] = df['tagline'].map(str) + ' ' + df['overview']
df.dropna(inplace=True)
df = df.sort_values(by=['popularity'], ascending=False)
df.info()

In [ ]:
df.head()

In [ ]:
df.iloc[4799].description

# Bangun Sistem Rekomendasi Film

Tahapan
- Pre Processing
- Feature Engineering
- Komputasi Doc Similarity
- Proses Retrieve
- proses rekomendasi film


## Kemiripan Dokumen / document similarity

Ada berbagai cara untuk menghitung kesamaan antara dua item dokumen. Salah satu ukuran yang paling banyak digunakan adalah __cosine similarity__ .

### Cosine Similarity

Cosine Similarity digunakan untuk menghitung skor numerik untuk menunjukkan kesamaan antara dua dokumen teks. Secara matematis, ini didefinisikan sebagai berikut:

$$ cosinus(x,y) = \frac{x. y^\intercal}{||x||.||y||} $$

In [ ]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['description']))
len(norm_corpus)

## Extrak TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

## Compute Pairwise Document Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

## mendapatkan title / judul movies

In [ ]:
movies_list = df['title'].values
movies_list, movies_list.shape

## Temukan Film Serupa Teratas untuk Contoh Film

Mari ambil __Minions__ film paling populer dari kerangka data di atas dan coba temukan film paling mirip yang dapat direkomendasikan

### ambil movie ID

In [ ]:
movie_idx = np.where(movies_list == 'Minions')[0][0]
movie_idx

### ambil similarities

In [ ]:
movie_similarities = doc_sim_df.iloc[movie_idx].values
movie_similarities

### Get top 5 similar movie IDs

In [ ]:
similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
similar_movie_idxs

### Get top 5 similar movies

In [ ]:
similar_movies = movies_list[similar_movie_idxs]
similar_movies

### Buat fungsi rekomendasi film untuk merekomendasikan 5 film serupa teratas untuk film apa pun


The movie title, movie title list and document similarity matrix dataframe akan diberikan sebagai input ke fungsi

In [ ]:
def movie_recommender(movie_title, movies=movies_list, doc_sims=doc_sim_df):
    # find movie id
    movie_idx = np.where(movies == movie_title)[0][0]
    # get movie similarities
    movie_similarities = doc_sims.iloc[movie_idx].values
    # get top 5 similar movie IDs
    similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
    # get top 5 movies
    similar_movies = movies[similar_movie_idxs]
    # return the top 5 movies
    return similar_movies

# misal cari rekomendasi untuk film film berikut

In [ ]:
popular_movies = ['Minions', 'Interstellar', 'Deadpool', 'Jurassic World', 'Pirates of the Caribbean: The Curse of the Black Pearl',
              'Dawn of the Planet of the Apes', 'The Hunger Games: Mockingjay - Part 1', 'Terminator Genisys', 
              'Captain America: Civil War', 'The Dark Knight', 'The Martian', 'Batman v Superman: Dawn of Justice', 
              'Pulp Fiction', 'The Godfather', 'The Shawshank Redemption', 'The Lord of the Rings: The Fellowship of the Ring',  
              'Harry Potter and the Chamber of Secrets', 'Star Wars', 'The Hobbit: The Battle of the Five Armies',
              'Iron Man']

In [ ]:
for movie in popular_movies:
    print('Movie:', movie)
    print('Top 5 recommended Movies:', movie_recommender(movie_title=movie, movies=movies_list, doc_sims=doc_sim_df))
    print()

# Rekomendasi Film (other method)

In [ ]:
# Gensim FastText
from gensim.models import FastText

tokenized_docs = [doc.split() for doc in norm_corpus]
ft_model = FastText(tokenized_docs, window=30, min_count=2, workers=4, sg=1)

# Hasilkan penyematan tingkat dokumen / Word Embedding

Model penyematan kata (word embedding) memberi kita penyematan untuk setiap kata, bagaimana kita dapat menggunakannya untuk tugas ML\DL hilirisasi? salah satu caranya adalah dengan meratakannya atau menggunakan model sekuensial. Pendekatan yang lebih sederhana adalah dengan rata-rata semua penyematan kata untuk kata-kata dalam dokumen dan menghasilkan penyematan tingkat dokumen dengan panjang tetap (fixed-length document level emebdding)

In [ ]:
def averaged_word2vec_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index_to_key)
    
    def average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0.
        
        for word in words:
            if word in vocabulary: 
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [ ]:
doc_vecs_ft = averaged_word2vec_vectorizer(tokenized_docs, ft_model, 100)
doc_vecs_ft.shape

# Dapatkan Rekomendasi Film

Memanfaatkan Cos Similarity untuk menghasilkan rekomendasi

In [ ]:
doc_sim = cosine_similarity(doc_vecs_ft)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

In [ ]:
for movie in popular_movies:
    print('Movie:', movie)
    print('Top 5 recommended Movies:', movie_recommender(movie_title=movie, movies=movies_list, doc_sims=doc_sim_df))
    print()